In [1]:
import pandas as pd
import time
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [2]:
def clean_text(text):
    return re.sub(r'\s+', ' ', text.strip().replace('\u202f', ''))

In [3]:
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

In [4]:
data = []

for page in range(1, 11):
    url = f"https://dakar-auto.com/senegal/location-de-voitures-19?page={page}"
    print(f"🔎 Scraping page {page}...")

    try:
        driver.get(url)
        time.sleep(2)

        annonces = driver.find_elements(By.CSS_SELECTOR, ".listings-cards__list-item")

        for annonce in annonces:
            try:
                # ✅ V1 : marque & V2 : année
                titre = clean_text(annonce.find_element(By.CSS_SELECTOR, "h2 a").text)
                parts = titre.split()
                marque = parts[0] if len(parts) > 0 else ""
                annee = parts[-1] if parts[-1].isdigit() and len(parts[-1]) == 4 else ""

                # ✅ V3 : prix
                prix = clean_text(annonce.find_element(By.CSS_SELECTOR, "h3").text)
                prix = prix.replace("F CFA", "").replace(" ", "")

                # ✅ V4 : adresse (ville + région)
                ville = ""
                region = ""
                try:
                    ville = clean_text(annonce.find_element(By.CLASS_NAME, "town-suburb").text)
                except:
                    pass
                try:
                    region = clean_text(annonce.find_element(By.CLASS_NAME, "province").text)
                except:
                    pass
                adresse = f"{ville}{region}".strip(",")

                # ✅ V5 : propriétaire (sans "Par ")
                try:
                    proprio = clean_text(annonce.find_element(By.CLASS_NAME, "time-author").text.replace("Par ", ""))
                except:
                    proprio = ""

                data.append({
                    "marque": marque,
                    "annee": annee,
                    "prix (FCFA)": prix,
                    "adresse": adresse,
                    "proprietaire": proprio
                })

            except Exception as e:
                print(f"⚠️ Erreur dans une annonce : {e}")
                continue

    except Exception as e:
        print(f"❌ Erreur chargement page {page} : {e}")
        continue

driver.quit()

🔎 Scraping page 1...
🔎 Scraping page 2...
🔎 Scraping page 3...
🔎 Scraping page 4...
🔎 Scraping page 5...
🔎 Scraping page 6...
🔎 Scraping page 7...
🔎 Scraping page 8...
🔎 Scraping page 9...
🔎 Scraping page 10...


In [5]:
df = pd.DataFrame(data)
pd.set_option("display.max_colwidth", None)
df.head(10)

,marque,annee,prix (FCFA),adresse,proprietaire
0,Citroen,2014,20000,"Médina,,Dakar",Chasseur Automobile
1,Peugeot,2013,30000,"Grand-Yoff,,Dakar",Chic Drive
2,Ford,2014,30000,"Sangalkam,,Dakar",Medoune Ndiaye
3,Ford,2015,35000,"Cité Keur gorgui,,Dakar",Pape Cheikh automobile
4,Toyota,2020,35000,"Mermoz Sacré-Coeur,,Dakar",LocAuto Sénégal Location de voiture
5,Hyundai,2015,20000,"Mermoz Sacré-Coeur,,Dakar",LocAuto Sénégal Location de voiture
6,Hyundai,2015,20000,"Mermoz Sacré-Coeur,,Dakar",LocAuto Sénégal Location de voiture
7,Toyota,2014,25000,"Mermoz Sacré-Coeur,,Dakar",LocAuto Sénégal Location de voiture
8,Ford,2014,30000,"Médina,,Dakar",Bakay Samassa
9,Kia,2015,25000,"Almadies,,Dakar",Moustapha SAMBA
